<div align="left">
    <img src="https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/imagenes/email_arriba.png" alt="Texto alternativo" width="50%">


#**¿Qué es Clasificación Multilabel?**

La clasificación multilabel (o clasificación multietiqueta) es un tipo de problema de aprendizaje supervisado en el que cada instancia (o ejemplo) puede pertenecer a más de una categoría o etiqueta simultáneamente. Esto difiere de la clasificación tradicional, en la que cada instancia pertenece a una única categoría.

#**Aula 01**

In [ ]:
import pandas as pd
!pip install scikit-multilearn
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.24.1
pd.set_option('display.float_format', '{:20,.2f}'.format)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
preguntas = pd.read_csv("stackoverflow_preguntas.csv", sep='|')
preguntas.sample(5)

#**Aula 02**

In [ ]:
preguntas.info()

In [ ]:
preguntas.Tags.unique()

In [ ]:
lista_de_tags = list()
for tags in preguntas.Tags.unique():
    for tag in tags.split():
        if tag not in lista_de_tags:
            lista_de_tags.append(tag)

print(lista_de_tags)

In [ ]:
lista = list()
for fila_tag in preguntas.Tags:
    if "list" in fila_tag:
        lista.append(1)
    else:
        lista.append(0)
preguntas["list"] = lista
preguntas.sample(5)

In [ ]:
def nueva_columna(lista_tags, dataframe, nombre_tags):
    for tag in lista_tags:
        columna = list()
        for fila_tag in dataframe[nombre_tags]:
            if tag in fila_tag:
                columna.append(1)
            else:
                columna.append(0)
        dataframe[tag] = columna

nueva_columna(lista_de_tags, preguntas, "Tags")
preguntas.sample(5)

#**Aula 03**

In [ ]:
from sklearn.model_selection import train_test_split

preguntas_train, preguntas_test, tags_train, tags_test = train_test_split(
    preguntas.Preguntas,
    preguntas.Tags
)

In [ ]:
lista_1 = [1,2]
lista_2 = [5,4]
lista_zip = zip(lista_1, lista_2)
print(list(lista_zip))

In [ ]:
lista_zip_tags = list(zip(preguntas[lista_de_tags[0]],
                     preguntas[lista_de_tags[1]],
                     preguntas[lista_de_tags[2]],
                     preguntas[lista_de_tags[3]],
                     preguntas[lista_de_tags[4]]))

preguntas["todas_tags"] = lista_zip_tags
preguntas.sample(5)

In [ ]:
from sklearn.model_selection import train_test_split

preguntas_train, preguntas_test, tags_train, tags_test = train_test_split(
    preguntas.Preguntas,
    preguntas.todas_tags,
    test_size = 0.2,
    random_state = 123
)

In [ ]:
preguntas_train.sample(5)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizar = TfidfVectorizer(max_features=5000, max_df=0.85)
vectorizar

In [ ]:
vectorizar.fit(preguntas.Preguntas)
preguntas_train_tfidf = vectorizar.transform(preguntas_train)
preguntas_test_tfidf = vectorizar.transform(preguntas_test)
print(preguntas_train_tfidf.shape)
print(preguntas_test_tfidf.shape)

#**Aula 04**

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

regresion_logistica = LogisticRegression()
clasificador_onevsrest = OneVsRestClassifier(regresion_logistica)
clasificador_onevsrest.fit(preguntas_train_tfidf, tags_train)

In [ ]:
type(tags_train)

In [ ]:
import numpy as np

tags_train_array = np.asarray(list(tags_train))
tags_test_array = np.asarray(list(tags_test))
print(tags_train_array)
print(type(tags_train_array))

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

regresion_logistica = LogisticRegression(solver = 'lbfgs')
clasificador_onevsrest = OneVsRestClassifier(regresion_logistica)
clasificador_onevsrest.fit(preguntas_train_tfidf, tags_train_array)
resultado_onevsrest = clasificador_onevsrest.score(preguntas_test_tfidf, tags_test_array)
print("Resultado {0: .2f}%".format(resultado_onevsrest*100))

#**Aula 05**

In [ ]:
from sklearn.metrics import hamming_loss

pronostico_onevsrest = clasificador_onevsrest.predict(preguntas_test_tfidf)
hamming_loss_onevsrest = hamming_loss(tags_test_array, pronostico_onevsrest)
print("Hamming Loss {0: .2f}".format(hamming_loss_onevsrest))

In [ ]:
preguntas.corr()

#**Aula 06**

In [ ]:
from skmultilearn.problem_transform import ClassifierChain

clasificador_cadena = ClassifierChain(regresion_logistica)
clasificador_cadena.fit(preguntas_train_tfidf, tags_train_array)
resultado_cadena = clasificador_cadena.score(preguntas_test_tfidf, tags_test_array)
pronostico_cadena = clasificador_cadena.predict(preguntas_test_tfidf)
hamming_loss_cadena = hamming_loss(tags_test_array, pronostico_cadena)
print("Hamming Loss {0: .2f}".format(hamming_loss_cadena))
print("Resultado {0: .2f}%".format(resultado_cadena*100))

In [ ]:
from skmultilearn.problem_transform import BinaryRelevance

clasificador_br = BinaryRelevance(regresion_logistica)
clasificador_br.fit(preguntas_train_tfidf, tags_train_array)
resultado_br = clasificador_br.score(preguntas_test_tfidf, tags_test_array)
pronostico_br = clasificador_br.predict(preguntas_test_tfidf)
hamming_loss_br = hamming_loss(tags_test_array, pronostico_br)
print("Hamming Loss {0: .2f}".format(hamming_loss_br))
print("Resultado {0: .2f}%".format(resultado_br*100))

#**Aula 07**

In [ ]:
from skmultilearn.adapt import MLkNN

clasificador_mlknn = MLkNN()
clasificador_mlknn.fit(preguntas_train_tfidf, tags_train_array)
resultado_mlknn = clasificador_mlknn.score(preguntas_test_tfidf, tags_test_array)
pronostico_mlknn = clasificador_mlknn.predict(preguntas_test_tfidf)
hamming_loss_mlknn = hamming_loss(tags_test_array, pronostico_mlknn)
print("Hamming Loss {0: .2f}".format(hamming_loss_mlknn))
print("Resultado {0: .2f}%".format(resultado_mlknn*100))

In [ ]:
resultados_clasificacion = pd.DataFrame()
resultados_clasificacion["Preguntas"] = preguntas_test.values
resultados_clasificacion["Tags"] = list(tags_test)
resultados_clasificacion["BR"] = list(pronostico_br.toarray())
resultados_clasificacion["Cadena"] = list(pronostico_cadena.toarray())
resultados_clasificacion["Mlknn"] = list(pronostico_mlknn.toarray())
print(lista_de_tags)
resultados_clasificacion.sample(5)

In [ ]:
resultados_clasificacion["sum Tags"] = list(map(sum, list(tags_test)))
resultados_clasificacion["sum BR"] = list(map(sum, list(pronostico_br.toarray())))
resultados_clasificacion["sum Cadena"] = list(map(sum, list(pronostico_cadena.toarray())))
resultados_clasificacion["sum Mlknn"] = list(map(sum, list(pronostico_mlknn.toarray())))
resultados_clasificacion.sample(5)

In [ ]:
print(lista_de_tags)
resultados_clasificacion[resultados_clasificacion["sum Cadena"]>1].sample(5)

[<div align="left">
    <img src="https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/imagenes/email_abajo.png" alt="Texto alternativo" width="50%">](https://bootcampxperience.com/)